<a href="https://colab.research.google.com/github/abhi-11nav/Rice_classification/blob/main/Rice_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# Importing libraries for buliding the model 

import tensorflow 
from tensorflow import keras 

from keras.layers import Dense, Flatten, Dropout 
from keras.models import Model 

from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.inception_v3 import InceptionV3


from glob import glob

# Visualization libraries 

import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns

## Data Processing

In [ ]:
# Importing splitfolder library for data seperation 

import splitfolders

ModuleNotFoundError: ignored

In [ ]:
# Split folders allows us to seperate training and testing data

"""
input_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice varieties'

output_path = '/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset'

splitfolders.ratio(input_path, output=output_path,seed=42, ratio=(.78,.22))
"""

In [ ]:
# Providing the train and validation paths 

train_path = "/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset/train"

validation_path = "/content/drive/MyDrive/Rice_Image_Dataset/Rice dataset/val"

In [ ]:
train_datagen = ImageDataGenerator(vertical_flip = True, horizontal_flip=True, zoom_range=0.2, rescale=1./255, shear_range=0.2)

validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_data = train_datagen.flow_from_directory(train_path, target_size=(224,224), class_mode='categorical')
validation_data = train_datagen.flow_from_directory(validation_path, target_size=(224,224),class_mode='categorical')

Found 56382 images belonging to 5 classes.
Found 15906 images belonging to 5 classes.


In [ ]:
training_data.class_indices

{'Arborio': 0, 'Basmati': 1, 'Ipsala': 2, 'Jasmine': 3, 'Karacadag': 4}

## Building the Model 

In [ ]:
# Calling the model function by specifying the input shape and retaining the imagenet weights

model_ = InceptionV3(include_top=False, weights='imagenet', input_shape = (224,224,3))

In [ ]:
# Setting the layers trainable property to False. So we only train the end layers of the model 

for layers in model_.layers:
  layers.trainable = False

In [ ]:
# Attaching additional flatten and output layer at the end of the model 

# Flatten layer 1 
flatten_layer = Flatten()(model_.output)

# Dense layer as the output layer with 5 units (output labels) and softmax activation function 
prediction_layer = Dense(5, activation="softmax")(flatten_layer)

In [ ]:
# Final model with input an output arguments passed

final_model = Model(inputs=model_.input, outputs=prediction_layer)

In [ ]:
# Summary of the model we are going to use on our data 

final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_94 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                             

In [ ]:
# Compiling the model 

final_model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
# Checking for GPU version 

!nvidia-smi 

Sat Aug 13 17:31:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    33W / 250W |   8761MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Fitting the model onto train and validation data

history = final_model.fit(training_data, validation_data = validation_data, epochs = 10, steps_per_epoch=len(training_data), validation_steps=len(validation_data))

KeyboardInterrupt: ignored

In [ ]:
final_model.save("model.h5")